In [2]:
import os
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [12]:
# Function to load and preprocess a video
def load_video(video_path, frames_per_video=10, img_height=64, img_width=64):
    frames = []
    cap = cv2.VideoCapture(video_path)
    while len(frames) < frames_per_video:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (img_width, img_height))
        frames.append(frame)
    cap.release()
    return frames

In [42]:
# Path to the directory containing subfolders with videos
data_dir = 'DCSASS Dataset'
class_names = os.listdir(data_dir)
num_classes = len(class_names)
print(class_names)

['Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Explosion', 'Fighting', 'Labels', 'Normal', 'RoadAccidents', 'Robbery', 'Shooting', 'Shoplifting', 'Stealing', 'Vandalism']


In [14]:
# Parameters
img_height, img_width = 64, 64
frames_per_video = 10

In [15]:
# Load videos and labels
videos = []
labels = []
for class_name in class_names:
    class_dir = os.path.join(data_dir, class_name)
    for root, _, files in os.walk(class_dir):
        for video_file in files:
            video_path = os.path.join(root, video_file)
            frames = load_video(video_path, frames_per_video, img_height, img_width)
            if len(frames) >= frames_per_video:
                videos.append(frames[:frames_per_video])
                labels.append(class_names.index(class_name))

In [16]:
# Convert videos and labels to numpy arrays
videos = np.array(videos)
labels = np.array(labels)

In [17]:
# Reshape the videos array to match the expected input shape for a Conv3D model
videos = videos.reshape(-1, frames_per_video, img_height, img_width, 3)

In [18]:
# Define the model
model = keras.Sequential([
    layers.Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=(frames_per_video, img_height, img_width, 3)),
    layers.MaxPooling3D(pool_size=(2, 2, 2)),
    layers.Conv3D(64, kernel_size=(3, 3, 3), activation='relu'),
    layers.MaxPooling3D(pool_size=(2, 2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])


In [19]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
# Train the model on the entire dataset
model.fit(videos, labels, epochs=20, batch_size=32)

Epoch 1/20
524/524 [==============================] - 544s 1s/step - loss: 2.7027 - accuracy: 0.4788
Epoch 2/20
524/524 [==============================] - 543s 1s/step - loss: 0.6240 - accuracy: 0.8172
Epoch 3/20
524/524 [==============================] - 578s 1s/step - loss: 0.3850 - accuracy: 0.8869
Epoch 4/20
524/524 [==============================] - 629s 1s/step - loss: 0.3083 - accuracy: 0.9075
Epoch 5/20
524/524 [==============================] - 620s 1s/step - loss: 0.2546 - accuracy: 0.9247
Epoch 6/20
524/524 [==============================] - 630s 1s/step - loss: 0.2507 - accuracy: 0.9241
Epoch 7/20
524/524 [==============================] - 627s 1s/step - loss: 0.2544 - accuracy: 0.9272
Epoch 8/20
524/524 [==============================] - 624s 1s/step - loss: 0.2071 - accuracy: 0.9417
Epoch 9/20
524/524 [==============================] - 597s 1s/step - loss: 0.2062 - accuracy: 0.9404
Epoch 10/20
524/524 [==============================] - 550s 1s/step - loss: 0.1965 - accura

In [21]:
# Save the trained model
model.save('video_classification_model2.h5')


c:\Users\sujal\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [22]:
# Make predictions
# predictions = model.predict("DCSASS Dataset/Explosion/Explosion004_x264.mp4")

In [53]:
# Evaluate the loaded model on the same dataset
loss, accuracy = model.evaluate(videos, labels)
print("Accuracy on the entire dataset:", accuracy)

524/524 [==============================] - 252s 471ms/step - loss: 0.0218 - accuracy: 0.9928
Accuracy on the entire dataset: 0.9928315281867981


In [2]:
loaded_model = keras.models.load_model('video_classification_model2.h5')

c:\Users\sujal\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [2]:
# import os
# import cv2
# import numpy as np
# from tensorflow import keras

# # Function to load and preprocess a single test video
# def load_test_video(video_path, frames_per_video=10, img_height=64, img_width=64):
#     frames = []
#     cap = cv2.VideoCapture(video_path)
#     while len(frames) < frames_per_video:
#         ret, frame = cap.read()
#         if not ret:
#             break
#         frame = cv2.resize(frame, (img_width, img_height))
#         frames.append(frame)
#     cap.release()
#     return np.array(frames)

# # Function to predict the class of a test video
# def predict_video_class(video_path, model, class_names, frames_per_video=10):
#     test_video = load_test_video(video_path, frames_per_video)
#     test_video = test_video.reshape(-1, frames_per_video, img_height, img_width, 3)
#     predictions = model.predict(test_video)
#     predicted_class_index = np.argmax(predictions)
#     predicted_class = class_names[predicted_class_index]
#     return predicted_class

# # Path to the directory containing subfolders with videos
# main_folder = 'DCSASS Dataset'
# class_names = os.listdir(main_folder)
# num_classes = len(class_names)

# # Load the saved model
# loaded_model = keras.models.load_model('video_classification_model2.h5')

# # Define the class names
# class_names = ['Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Explosion', 'Fighting', 'Labels', 'Normal', 'RoadAccidents', 'Robbery', 'Shooting', 'Shoplifting', 'Stealing', 'Vandalism']  # Modify this based on your actual class names

# # Parameters for the test videos
# img_height, img_width = 64, 64
# frames_per_video = 10

# # Function to process videos in a folder and its subfolders
# def process_videos_in_folder(folder_path):
#     for root, _, files in os.walk(folder_path):
#         for file in files:
#             video_path = os.path.join(root, file)
#             if video_path.endswith(('.mp4', '.avi', '.mpg', '.mov', '.flv', '.wmv')):
#                 predicted_class = predict_video_class(video_path, loaded_model, class_names, frames_per_video)
#                 print("Video:", video_path)
#                 print("Predicted class:", predicted_class)

# # Process videos in the main folder and its subfolders
# process_videos_in_folder(main_folder)

# # Additionally, if you want to process videos in the folder with only videos
# videos_folder = 'DCSASS Dataset/Shooting/Shooting048_x264.mp4/Shooting048_x264_1.mp4'
# process_videos_in_folder(videos_folder)


In [5]:
import os
import cv2
import numpy as np
from tensorflow import keras

# Function to load and preprocess a single test video
def load_test_video(video_path, frames_per_video=10, img_height=64, img_width=64):
    frames = []
    cap = cv2.VideoCapture(video_path)
    while len(frames) < frames_per_video:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (img_width, img_height))
        frames.append(frame)
    cap.release()
    return np.array(frames)

# Function to predict the class of a test video
def predict_video_class(video_path, model, class_names, frames_per_video=10):
    test_video = load_test_video(video_path, frames_per_video)
    test_video = test_video.reshape(-1, frames_per_video, img_height, img_width, 3)
    predictions = model.predict(test_video)
    predicted_class_index = np.argmax(predictions)
    predicted_class = class_names[predicted_class_index]
    return predicted_class

loaded_model = keras.models.load_model('video_classification_model2.h5')

class_names = ['Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Explosion', 'Fighting', 'Labels', 'Normal', 'RoadAccidents', 'Robbery', 'Shooting', 'Shoplifting', 'Stealing', 'Vandalism']  # Modify this based on your actual class names

img_height, img_width = 64, 64
frames_per_video = 10

In [6]:
test_video_path = 'backend/DCSASS Dataset/Arrest/Arrest001_x264.mp4/Arrest001_x264_0.mp4'

predicted_class = predict_video_class(test_video_path, loaded_model, class_names)

# Print the predicted class
print("Predicted class:", predicted_class)

ValueError: math domain error